In [44]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_chroma import Chroma  # 从 langchain_chroma 中引用 Chroma 类
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)

persist_directory = './langchain-chroma'

# 初始化嵌入
embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)

# 使用 embedding_function 参数初始化 Chroma
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embed
)

chat = ChatZhipuAI(
    model="glm-4-flash",
    temperature=0.5,
    api_key =  "5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io"
)

from langchain.chains import RetrievalQA
question = "这本书最重要的主题是?"
qa_chain = RetrievalQA.from_chain_type(
    chat, # 大模型的选用
    retriever=vectordb.as_retriever() # 向量数据库的调用
)
result = qa_chain({"query": question}) # 输入问题
print(result["result"]) # 获取回复

这本书最重要的主题是 **深度学习**，特别是 **动手学习深度学习**。它涵盖了深度学习的各个方面，包括：

* **深度学习基础**：介绍机器学习、神经网络、深度学习等基础知识。
* **深度学习模型**：详细讲解各种深度学习模型，如线性神经网络、卷积神经网络、循环神经网络等。
* **深度学习实践**：通过动手实践，学习如何使用深度学习框架（如PyTorch、TensorFlow等）构建和训练深度学习模型。
* **深度学习应用**：展示深度学习在计算机视觉、自然语言处理等领域的应用。

总而言之，这本书旨在帮助读者全面了解深度学习，并通过实践掌握深度学习技能。


In [1]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_chroma import Chroma  # 从 langchain_chroma 中引用 Chroma 类
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
splits = text_splitter.split_documents(docs)

persist_directory = './langchain-chroma'

# 初始化嵌入
embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)

# 使用 embedding_function 参数初始化 Chroma
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embed
)

chat = ChatZhipuAI(
    model="glm-4-flash",
    temperature=0.5,
    api_key =  "5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io"
)

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Build prompt
template = """请使用以下上下文信息回答最后的问题。
如果您不知道答案，就直接说您不知道，不要试图编造答案。
回答最多使用三句话。请尽可能简洁地回答。最后一定要说“谢谢提问！”。
上下文：{context}
问题：{question}
有帮助的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Run chain
qa_chain = RetrievalQA.from_chain_type(
    chat,
    retriever=vectordb.as_retriever(),
    return_source_documents=True, # 这个可有可无，只是返回找到的源文件内容
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "这本书最重要的主题是?"
result = qa_chain({"query": question})
print(result["result"])

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\will\AppData\Local\Temp\ipykernel_9588\3337692333.py:60: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


这本书最重要的主题是深度学习，涵盖了从基础知识到高级应用，包括各种模型、算法和工具。

谢谢提问！


In [2]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_community.embeddings import ZhipuAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# 文件导入
loader = WebBaseLoader("https://zh.d2l.ai/")
docs = loader.load()

# 文本切分
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)
splits = text_splitter.split_documents(docs)

# 初始化嵌入
persist_directory = './langchain-chroma'

# 初始化嵌入
embed = ZhipuAIEmbeddings(model="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embed
)

# 初始化聊天模型
chat = ChatZhipuAI(
    model="glm-4-flash",
    temperature=0.5,
    api_key =  "5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io"
)

# 初始化内存
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 创建 ConversationalRetrievalChain
retriever = vectordb.as_retriever()
qa = ConversationalRetrievalChain.from_llm(
    chat,
    retriever=retriever,
    memory=memory
)

# 第一次提问，明确告诉模型要记住的信息
question = "这本书有讲到关于计算机视觉的内容吗?不超过100字。"
result = qa.invoke({"question": question})
print(result['answer'])

# 第二次提问，模型会基于记忆回答
question = "请用几句话介绍一下这个主题可以吗？不超过100字。"
result = qa.invoke({"question": question})
print(result['answer'])

C:\Users\will\AppData\Local\Temp\ipykernel_9588\4246657298.py:41: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


是的，这本书涵盖了计算机视觉的多个方面，包括图像增广、微调、目标检测、语义分割、风格迁移等，并介绍了相关数据集和模型，例如SSD、R-CNN、全卷积网络等。
根据提供的目录内容，可以得出以下结论：

* **图像处理技术**: 
    * 目录中涵盖了图像增广、微调等技术，这些技术用于改进图像数据，提高模型性能。
    * 此外，还介绍了目标检测、语义分割等技术，这些技术直接与图像处理相关。
* **目标检测**:
    * 目录中详细介绍了目标检测的多个方面，包括：
        * 锚框和多尺度目标检测
        * 单发多框检测（SSD）
        * 区域卷积神经网络（R-CNN）系列
    * 这些内容涵盖了目标检测的基本概念、方法和应用。
* **语义分割**:
    * 目录中介绍了语义分割的基本概念和数据集，例如：
        * 语义分割和数据集
        * 转置卷积
        * 全卷积网络
    * 这些内容为理解语义分割提供了基础。
* **相关数据集和模型**:
    * 目录中提到了多个数据集，例如：
        * 目标检测数据集
        * 语义分割数据集
        * CIFAR-10 数据集
        * ImageNet Dogs 数据集
    * 这些数据集用于训练和测试模型。
    * 目录中还提到了一些模型，例如：
        * SSD 模型
        * R-CNN 模型系列
        * 全卷积网络
    * 这些模型是计算机视觉领域常用的模型。

综上所述，这本书确实主要介绍了计算机视觉的多个方面，包括图像处理技术、目标检测、语义分割等，并涉及相关数据集和模型。
